## Implement LMS, NLMS and RLS filters and compare performance for noise filtering

In [1]:
import numpy as np
from scipy.io.wavfile import read, write
import time


In [2]:
# load data and normalize
fs1, d = read('data/Noise.wav')
d = d/max(d)
# Signal
fs1, x = read('data/Music.wav')
x = x/max(x)
# Noisy signal
Fs, u = read('data/Noisy_Music.wav')
u_max = max(u)
u = u/u_max

In [3]:
def calc_power(signal):
    return np.mean(np.abs(signal)**2)

def calc_SNR(signal, noise):
    P_sig = calc_power(signal)
    P_noise = calc_power(noise)
    return 10*np.log10(P_sig/P_noise)

In [13]:
# implement LMS
LMS_start = time.time()

# setup filter parameters
N = len(d)
M = 12
# to find the appropriate step size, we need to estimate the power of the signal
P_u = 1/N * np.sum(u**2)
# mu is found with (mu = alpha / P_u), where alpha is a small constant between 0.01 and 0.1. Stability is guaranteed if mu < 2 / P_u
mu = 0.01 / P_u

w_lms = np.zeros(M) # initialize filter weights at 0
u_pad = np.pad(u, (M-1, 0), 'constant') # pad noisy signal with zeros
y = np.zeros(N) # initialize output signal

for n in range(0, N):
    u_samples = u_pad[n:n+M] # grab M amount of samples from observed signal (-1 because we want to go backwards)
    e = d[n] - np.matmul(w_lms,u_samples) # calculate error
    w_lms = w_lms + mu*e*u_samples # update weights
    y[n] = np.matmul(w_lms, u_samples) # apply weights to observed signal
LMS_done = time.time()
    
# calculate and save output signal
LMS_output = u - y
write(f"lms_filtered.wav", Fs, (LMS_output*u_max).astype(np.int16))

#evaluate LMS with time and SNR
print(f"LMS took {LMS_done - LMS_start:.2f} seconds")
SNR_before = calc_SNR(x, u-x)
print(f"SNR before filtering: {SNR_before:.2f} dB")
SNR_after = calc_SNR(x, LMS_output - x)
print(f"SNR after filtering with LMS: {SNR_after:.2f} dB")


LMS took 1.86 seconds
SNR before filtering: -0.09 dB
SNR after filtering with LMS: 1.67 dB


In [5]:
# implement NLMS
NLMS_start = time.time()

# setup filter parameters
N = len(d) # number of samples
M = 12 # filter order
mu = 1 # for some reason I get better results with a fixed step size of 1 instead of relating it to the power of the input signal
w_nlms = np.zeros(M) # initialize filter weights at 0
u_pad = np.pad(u, (M-1, 0), 'constant') # pad noisy signal with zeros
y = np.zeros(N) # initialize output signal
epsilon = 0.0001 # small constant to avoid division by zero when updating step size, mu

for n in range(0, N):
    u_samples = u_pad[n:n+M] # grab M amount of samples from observed signal
    mu_adapt = mu / (epsilon + np.linalg.norm(u_samples, ord=2)**2) # update step size by normalizing with the power of the input signal
    e = d[n] - w_nlms@u_samples # calculate error
    w_nlms = w_nlms + mu_adapt*e*u_samples # update weights
    y[n] = w_nlms@u_samples # apply weights to observed signal
NLMS_done = time.time()

# calculate and save output signal
NLMS_output = u - y
write(f"nlms_filtered.wav", Fs, (NLMS_output*u_max).astype(np.int16))

#evaluate LMS with time and SNR
print(f"NLMS took {NLMS_done - NLMS_start:.2f} seconds")
SNR_before = calc_SNR(x, u-x)
print(f"SNR before filtering: {SNR_before:.2f} dB")
SNR_after = calc_SNR(x, NLMS_output - x)
print(f"SNR after filtering with NLMS: {SNR_after:.2f} dB")

NLMS took 3.04 seconds
SNR before filtering: -0.09 dB
SNR after filtering with NLMS: 3.21 dB


In [6]:
# implement RLS
RLS_start = time.time()

# setup filter parameters
N = len(d) # number of samples
M = 12 # filter order
delta = 0.001 # weight of initial filter coefficients
lmbd = 0.16 # forgetting factor
p_inversecorr = 1/delta*np.identity(M) # initialize inverse correlation matrix
w_rls = np.zeros(M) # initialize filter weights at 0
u_pad = np.pad(u, (M-1, 0), 'constant') # pad noisy signal with zeros
y = np.zeros(N) # initialize output signal

for n in range(0, N):
    u_samples = u_pad[n:n+M] # grab M amount of samples from observed signal
    pi_func = p_inversecorr@u_samples # calculate helper vector used to update gain vector K
    k_gain = pi_func / (lmbd + u_samples@pi_func) # calculate gain vector
    priori_error = d[n] - w_rls@u_samples # calculate priori error
    w_rls = w_rls + k_gain*priori_error # update filter weights
    p_inversecorr = p_inversecorr/lmbd - np.outer(k_gain, u_samples@p_inversecorr)/lmbd# update inverse correlation matrix
    
    y[n] = w_rls@u_samples # apply weights to observed signal
RLS_done = time.time()

# calculate and save output signal
RLS_output = u - y
write(f"RLS_filtered.wav", Fs, (RLS_output*u_max).astype(np.int16))

#evaluate LMS with time and SNR
print(f"RLS took {RLS_done - RLS_start:.2f} seconds")
SNR_before = calc_SNR(x, u-x)
print(f"SNR before filtering: {SNR_before:.2f} dB")
SNR_after = calc_SNR(x, RLS_output - x)
print(f"SNR after filtering with RLS: {SNR_after:.2f} dB")

RLS took 5.97 seconds
SNR before filtering: -0.09 dB
SNR after filtering with RLS: 3.22 dB
